huggingface/transformers

typical workflows  

initiate a model with model_type and model_name, e.g.,  
'distilbert', 'distilbert-base-uncased-distilled-squad' in https://www.kaggle.com/jonathanbesomi/question-answering-starter-pack  
'roberta', 'roberta-large' in https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing  
respectivly

check model_doc for the selected model, especially for    
overview  
tokenizer to use  
class transformers.{model_type}Model, outputting raw hidden-states without any specific head on top  
on which forward method about inputs required

EXAMPLES  
for roberta, check https://huggingface.co/transformers/model_doc/roberta.html  
we may know roberta use Byte-level BPE, token_type_ids not required, instead separate segments with \</s>  
for BERT, https://huggingface.co/transformers/model_doc/bert.html  
based on WordPiece, check forward about inputs required

tokenizers generates inputs for models, forward pass,  
but we need to generates targets for loss, backward propagation  
in some cases, we also need to generate supplement information for scoring

EXAMPLE  
check https://www.kaggle.com/sblroid/tweet-sentiment-extraction-fromscratch/

More on  
*https://huggingface.co/transformers/training.html  
https://huggingface.co/transformers/custom_datasets.html

Documentation transformers package

about common methods of classes are found in MAIN CLASSES tab on the left

for example, about tokenizers are in https://huggingface.co/transformers/main_classes/tokenizer.html  
\__call__ method is a general method used in a number of tokenizers  

but, since tokenizers are inherited from PreTrainedTokenizerBase,  
where some common methods such as tokenizer.from_pretrained() are defined,  
we may also need to refer INTERNAL HELPERS/Utilities for Tokenizers on the left  
https://huggingface.co/transformers/internal/tokenization_utils.html#pretrainedtokenizerbase

In [ ]:
CheetSheat
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')
encoding = tokenizer(df_train.text[0:5].tolist(), df_train.selected_text[0:5].tolist(), return_tensors='pt', padding=True, truncation=True)
model(**encoding)

***

Tokenizers Theory Summary  

https://huggingface.co/transformers/tokenizer_summary.html

naive tokenizations generate a too big vocabulary so subword tokenization is used instaed

Depending on the rules we apply for tokenizing a text, a different tokenized output is generated for the same text. A pretrained model 
only performs properly if you feed it an input that was tokenized with the same rules that were used to tokenize its training data.

Three main types: BPE, WordPiece, SentencePiece

BPE  
first perform naive pre-tokenization  
create a base vocabulary of all symbols and learns merge rules  
: 'hug' -> 'h' 'u' 'g' -> 'h' 'ug'  
does to until the desired size

Byte-level BPE  
for roberta

WordPiece  
WordPiece is the subword tokenization algorithm used for BERT, DistilBERT, and Electra.  
very similar to BPE  
WordPiece is slightly different to BPE in that it evaluates what it loses by merging two symbols to make ensure it’s worth it.

SentencePiece  
All tokenization algorithms described so far have the same problem: It is assumed that the input text uses spaces to separate words. However, not all languages use spaces to separate words.  
All transformers models in the library that use SentencePiece use it in combination with unigram. Examples of models using SentencePiece are ALBERT, XLNet, Marian, and T5.

transformers.TokernizerFast in use 

tokenizer(text, [text], ... ,return_offsets_mapping=True)  
return_offsets_mapping=True only available on fast tokenizerz inheriting from PreTrainedTokenizerFast

NOTE THAT PreTrainedTokenizerFast is the base class for all fast tokenizers wrapping HuggingFace tokenizers library

the required files for offline tokenizers package can be obtained by  
transformers.BertTokenizerFast.save_pretrained(PATH_TO_DIR)  
which generates vocab.txt, merges.txt, etc.

tokenizers in use  

encoding = tokenizer.encode(STR)  
encoding.ids  
encoding.offsets  

no need to specify keyword for offsets  
simpler to use

about input_ids, offsets generating from tokenizer

tokens and inputs_ids, 1-1 correspondence  
EXAMPLE  
I am a student -> [101, 1045, 2572, 1037, 3076, 102], by BertWordPieceTokenization  
each element is the dictionary key for the token  
101 and 102 are special tokens

input_ids and offsets, 1-1 correspondence  
offsets[i] = (char_start, char_end) for the token of input_ids[i]  
original_text[char_start: char_end] corresponds to a token since indices in a offset are with respect to the original text string,  
EXAMPLE  
i am a student -> [(0, 0), (0, 1), (2, 4), (5, 6), (7, 14), (0, 0)]  
offsets[2] = (2, 4)  
input_ids[2] = 2572  
'i am a student'[2: 4] = 'am'

by looking offset, we can get the information of how tokens were separated, without encoding selected_text  
without offset, for locating selected_text in input_ids, we should identify ids-token matches  
EXAMPLE  
[4997, 102, 17111, 2080, 6517, 1045, 2097] input_ids of tweet, [6517, 1045] ids of selected_text  
but without offsets, we should encode selected_text and compare ids to locate selected_text in input_ids

useful CHEATSHEET  
https://www.kaggle.com/debanga/huggingface-tokenizers-cheat-sheet

TIPS  

tokenizer receives  
a STR  
or a list of STR  
or a pair of lists of STR  

EXAMPLE  
encoding1 = tokenizer(df_train.text[0:5].tolist(), df_train.selected_text[0:5].tolist(), return_tensors='pt', padding=True, truncation=True)

transformers.Tokenizer module vs tokenizers package

transformers.BertTokenizer.from_pretrained('bert-base-uncased' or path)  
many other pretrained models are listed in  
https://huggingface.co/transformers/_modules/transformers/tokenization_bert.html

tokenizers.BertWordPieceTokenizer(
    f"{BERT_PATH}/vocab.txt", 
    lowercase=True
)  
vocab.txt can be obtained by  
https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.save_pretrained

Since tweet-sentiment-extraction notebooks cannot access to the Internet,  
kernels used tokenizers package with save_pretrained  

We may use transformers.TokenizerFast instead, but it is a wrapper for the tokenizers package, so may be slower  

***

transformers.roberta  
https://huggingface.co/transformers/model_doc/roberta.html  

RoBERTa has the same architecture as BERT, but uses a byte-level BPE as a tokenizer (same as GPT-2) and uses a different pretraining scheme.

RoBERTa doesn’t have token_type_ids, you don’t need to indicate which token belongs to which segment. Just separate your segments with the separation token tokenizer.sep_token (or </s>)

Checking for the architecture  
roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH)  
roberta
shows that 
embedding  
encoder comprised of 12 RobertaLayer  
pooler of Dense with tanh activation

INPUT for roberta  
forward(self, input_ids, attention_mask, token_type_ids, ...)  
input_ids:  
Indices of input sequence tokens in the vocabulary.  
attention_mask:  
Mask to avoid performing attention on padding token indices.  
https://huggingface.co/transformers/glossary.html#attention-mask  
This can then be converted into a tensor in PyTorch or TensorFlow. The attention mask is a binary tensor indicating the position of the padded indices so that the model does not attend to them.  
token_type_ids:  
https://huggingface.co/transformers/glossary.html#token-type-ids  
Some models’ purpose is to do sequence classification or question answering. These require two different sequences to be joined in a single “input_ids” entry, which usually is performed with the help of special tokens, such as the classifier ([CLS]) and separator ([SEP]) tokens. For example, the BERT model builds its two sequence input as such:  
The first sequence, the “context” used for the question, has all its tokens represented by a 0, whereas the second sequence, corresponding to the “question”, has all its tokens represented by a 1.

***

simpletransformers

transformers vs simpletransformers  
simpletransformers is the package built on transformers for easy use  
model_type, model_name arguments are identical  
?the form required for train data is also identical  

Usual steps for using simpletransformrs  
Preparing train data as json of the format
https://simpletransformers.ai/docs/qa-data-formats/

Initiate a model with model_type and model_name  
model_type : bert, roberta, distilled, etc.  
model_name: specifies the exact architecture and trained weights to use  
https://simpletransformers.ai/docs/usage/#task-specific-models  
https://simpletransformers.ai/docs/qa-model/

model.train_model(PATH_TO_JSON)

***

spaCy

In [ ]:
HOW to use SPACY  
codes similar here  
https://yujuwon.tistory.com/entry/spaCy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-Training-Models

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
    
https://www.kaggle.com/rohitsingh9990/ner-training-using-spacy-ensemble?scriptVersionId=0

In [ ]:
training data for entity recognition  
(text, {'entities': [(start, end, class)]})  
text[start:end] -> entities in text  

Example  
('I like London and Berlin.', { 'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')] })

The belows are from https://www.kaggle.com/sblroid/tweet-sentiment-extraction-eda

In [ ]:
def get_training_data(sentiment):
    '''
    Returns Trainong data in the format needed to train spacy NER
    '''
    train_data = []
    for index, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
            # (text, selected_text)
            # selected_text = text[start:end]
    return train_data

In [ ]:
def get_model_out_path(sentiment):
    if sentiment == 'positive':
        model_out_path = 'models/model_pos'
    elif sentiment == 'negative':
        model_out_path = 'models/model_neg'
    else:
        model_out_path = None
    return model_out_path

In [ ]:
def train(train_data, output_dir, n_iter=20, model=None):
    if model is not None:
        nlp = spacy.load(output_dir)
        print(f"Loaded model '{model}'")
    else:
        nlp = spacy.blank('en')
        print("Created blank 'en' model")
        
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe('ner')
    # as a result, a pipe in nlp and we got ner to add labels
        
    for _, annotations in train_data:  # annotations : Dict
        for ent in annotations.get('entities'):  # 'entities': [[start, end, 'selected_text']]
            ner.add_label(ent[2])  # ent[2]: 'selected_text'
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()
            
        for itn in range(n_iter):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4, 500, 1.001))
            # minibatch : given train_data, returns a generator yielding batches of the given size
            # compounding(4, 500, 1.001):start 4, next 4 * 1.001, end 500
            # in this example, size=4 and size=compounding(4, 500, 1.001) generates the same result
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                # texts: a tuple of text of size 4 
                # annotations: a tuple of dict of size 4
                nlp.update(texts, annotations, drop=0.5, losses=losses)
            print('Losses', losses)
    save_model(output_dir, nlp, 'st_ner')